In [1]:
import numpy as np
import statistics as stats
from scipy.stats import norm
# add here the libraries you need

# Discrete Time Markov Chains - Part 2
This is an exercise notebook on DTMCs. 

Remember to revise of the lecture on DTMC simulation before attempting to solve it!
In order to complete this notebook, you need the models implemented in Part 1 notebook on DTMC.

### 1. Simulation of DTMC
Write a method that simulates a DTMC for `n` steps, where `n` is a parameter of the method, and returns the whole trajectory as output.

In [2]:
def sample_discrete(N,p):
    """Computes N samples from discrete probability vector p"""
    n = len(p)
    S = np.cumsum(p)
    U = np.random.uniform(low=0.0, high=1.0, size=N)
    sampled_indexes = np.empty(N)
    for j in range(N):
        for i in range(n):
            if S[i] > U[j]:
                sampled_indexes[j] = i
                break
    return sampled_indexes.astype(int)

def simulate_dtmc(transition_model, initial_prob_vector, n):
    """Given a stocastic matrix, an initial probability vector and the number of steps, simulates a trajectory of the DTMC"""
    assert n > -1, "Negative amount of steps."
    current_state = sample_discrete(1,initial_prob_vector)[0]
    trajectory = [current_state]
    if n == 0:
        return trajectory
    for i in range(n):
        current_state = sample_discrete(1,transition_model[current_state,:])[0]
        trajectory.append(current_state)
    return trajectory

In [3]:
# Model of a birth-death chain created in the previous exercises:
def check_vectors(N,p,q):
    """Function that checks if the vectors for a general birth-death chain of maximum population N are valid"""
    if len(p) != N or len(q) != N: # vectors must be of size N
        return False
    for i in range(N):
        if p[i] + q[i] > 1: # For every state, the sum of the probabilities should not exceed 1
            return False
    return True

def build_transition_birth_death(N, p, q):
    """Function that builds a transition matrix for a general birth-death chain of maximum population N, 
    given the vector of probabilities of birth and death"""
    assert check_vectors(N,p,q), "Wrong vectors."
    assert N > 0, "Negative max population"
    
    transition_birth_death = np.zeros((N,N))
    
    transition_birth_death[0,0] = 1 - p[0]
    transition_birth_death[0,1] = p[0]
    
    for i in range(1,N-1):
        transition_birth_death[i,i-1] = q[i-1]
        transition_birth_death[i,i+1] = p[i]
        transition_birth_death[i,i] = 1 - p[i] - q[i-1]
        
    transition_birth_death[N-1,N-2] = q[N-2]
    transition_birth_death[N-1,N-1] = 1 - q[N-2]
    
    return transition_birth_death

N = 5
p = np.array([0.2,0.3,0.5,0.6,0.4])
q = np.array([0.6,0.5,0.3,0.2,0.5])

transition_birth_death = build_transition_birth_death(N,p,q)
initial_prob_vector = np.array([0.,0.,1.,0.,0.])

In [4]:
trajectory = simulate_dtmc(transition_birth_death, initial_prob_vector, 10)
print("Simulation of 10 steps:", trajectory)

Simulation of 10 steps: [2, 3, 4, 4, 3, 4, 4, 3, 3, 2, 3]


### 2. Statistical analysis
Write methods for:
- 2.1. computing the average of a function `f` of the state space, at time step `n`.
- 2.2. computing the probability of reaching a target region `A` of the state space by time step `n`.

Both methods should use simulation, and return an estimate and a confidence interval at a specified confidence level `alpha` (0.95% by default).

In [5]:
def f(trajectory):
    """Returns the number of times that the state number 3 occurred in the trajectory."""
    acc = 0
    for state in trajectory:
        acc += 1 if state == 3 else 0
    return acc

In [18]:
def confidence_interval(samples, alpha=0.95):
    mean = np.mean(samples)
    std_dev = np.std(samples, ddof=1)
    perc = (1+alpha)/2
    q = norm.ppf(perc)
    epsilon = q * std_dev / np.sqrt(len(samples))
    rhs = mean + epsilon
    lhs = mean - epsilon
    return (lhs,rhs)

In [21]:
%%time
def f_average(f, transition_model, initial_prob_vector, n, alpha = 0.95):
    """Computes the average of f by simulation of DTMC of n steps. Arguments: function, stpcastic matrix, initial
    probability vector, number of steps and the confidence level. Outputs: the average and the confidence interval."""
    N_SIM = 100000 # number of simulations
    samples = []
    for i in range(N_SIM):
        trajectory = simulate_dtmc(transition_model, initial_prob_vector, n)
        y = f(trajectory)
        samples.append(y)
    return (np.mean(samples),confidence_interval(samples, alpha))

def target_in_n(transition_model, initial_prob_vector, target, n):
    """Computes the probability, using simulation, of reaching a target region by time step n."""
    pass

f_average(f,transition_birth_death,initial_prob_vector,10)

Wall time: 10.9 s


(1.58375, (1.5746536817696737, 1.5928463182303263))

### 3. Branching chain
Consider a population, in which each individual at each
generation independently gives birth to $k$ individuals with
probability $p_k$. These will be the members of the next
generation. Assume $k\in\{-1, 0,1,2\}$. The population is initial compused of two individuals Adam and Eve.

In [110]:
norm.ppf(0.975)

1.959963984540054

Assume now that $p_0 = p_1 = p_2 = (1-p_{-1})/3$. Estimate the average and the confidence interval of the probability of the population to become extinct for increasing values of $p_{-1}$.